# Using Selenium to Test User Interactions

Where were we at the end of the last chapter? Let’s rerun the test and find out:

In [1]:
%cd ../examples/superlists/

/home/thunder/Documents/work/codeguild2015/code_guild/wk9/examples/superlists


In [2]:
!python3 functional_tests.py

F
FAIL: test_can_start_a_list_and_retrieve_it_later (__main__.NewVisitorTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "functional_tests.py", line 20, in test_can_start_a_list_and_retrieve_it_later
    self.assertIn('To-Do', self.browser.title)  #5
AssertionError: 'To-Do' not found in 'Problem loading page'

----------------------------------------------------------------------
Ran 1 test in 2.356s

FAILED (failures=1)


Did you try it, and get an error saying Problem loading page or Unable to connect? So did I. It’s because we forgot to spin up the dev server first using manage.py runserver. Do that, and you’ll get the failure message we’re after.

>One of the great things about TDD is that you never have to worry about forgetting what to do next—just rerun your tests and they will tell you what you need to work on.

“Finish the test”, it says, so let’s do just that! Open up functional_tests.py and we’ll extend our FT:


In [6]:
%%writefile functional_tests.py

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import unittest

class NewVisitorTest(unittest.TestCase):
    
    def setUp(self):
        self.browser = webdriver.Firefox()
        self.browser.implicitly_wait(3)
        
    def tearDown(self):
        self.browser.quit()
        
    def test_can_start_a_list_and_retrieve_it_later(self):
        # Edith has heard about a cool new online to-do app. She goes 
        # to check out its homepage
        self.browser.get('http://localhost:8000')
        
        # She notices the page title and header mention to-do lists
        self.assertIn('To-Do', self.browser.title)
        header_text = self.browser.find_element_by_tag_name('h1').text
        self.assertIn('To-Do', header_text)
        
        # She is invited to enter a to-do item straight away
        inputbox = self.browser.find_element_by_id('id_new_item')
        self.assertEqual(
            inputbox.get_attribute('placeholder'),
            'Enter a to-do item'
        )
        
        # She types "Buy peacock feathers" into a text box (Edith's hobby)
        # is tying fly-fishing lures)
        inputbox.send_keys('Buy peacock feathers')
        
        # When she hits enter, the page updates, and now the page lists
        # "1: Buy peacock feathers" as an item in a to-do list table
        inputbox.send_keys(Keys.ENTER)
        
        table = self.browser.find_element_by_id('id_list_table')
        rows = table.find_elements_by_tag_name('tr')
        self.assertTrue(
            any(row.text == '1: Buy peacock feathers' for row in rows)
        )
        
        # There is still a text box inviting her to add another item. She
        # enters "Use peacock feathers to make a fly" (Edith is very
        # methodical)
        self.fail('Finish the test!')
        
        # The page updates again, and now shows both items on her list

        # Edith wonders whether the site will remember her list. Then she sees
        # that the site has generated a unique URL for her -- there is some
        # explanatory text to that effect.

        # She visits that URL - her to-do list is still there.

        # Satisfied, she goes back to sleep

if __name__ == '__main__':  
    unittest.main(warnings='ignore')  

Overwriting functional_tests.py


We’re using several of the methods that Selenium provides to examine web pages: `find_element_by_tag_name`, `find_element_by_id`, and `find_elements_by_tag_name` (notice the extra s, which means it will return several elements rather than just one). We also use send_keys, which is Selenium’s way of typing into input elements. You’ll also see the Keys class (don’t forget to import it), which lets us send special keys like `Enter`, but also modifiers like `Ctrl`.

>Watch out for the difference between the Selenium find_element_by... and find_elements_by... functions. One returns an element, and raises an exception if it can’t find it, whereas the other returns a list, which may be empty.

Also, just look at that `any` function. It’s a little-known Python built-in. I don’t even need to explain it, do I? Python is such a joy.

Although, if you’re one of my readers who doesn’t know Python, what’s happening inside the `any` is a generator expression, which is like a list comprehension but awesomer. You need to read up on this. If you Google it, you’ll find Guido himself explaining it nicely. Come back and tell me that’s not pure joy!

Let’s see how it gets on:


In [8]:
!python3 functional_tests.py

E
ERROR: test_can_start_a_list_and_retrieve_it_later (__main__.NewVisitorTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "functional_tests.py", line 22, in test_can_start_a_list_and_retrieve_it_later
    header_text = self.browser.find_element_by_tag_name('h1').text
  File "/home/thunder/anaconda3/envs/django_env/lib/python3.5/site-packages/selenium/webdriver/remote/webdriver.py", line 354, in find_element_by_tag_name
    return self.find_element(by=By.TAG_NAME, value=name)
  File "/home/thunder/anaconda3/envs/django_env/lib/python3.5/site-packages/selenium/webdriver/remote/webdriver.py", line 712, in find_element
    {'using': by, 'value': value})['value']
  File "/home/thunder/anaconda3/envs/django_env/lib/python3.5/site-packages/selenium/webdriver/remote/webdriver.py", line 201, in execute
    self.error_handler.check_response(response)
  File "/home/thunder/anaconda3/envs/django_env/lib/python3.5/site-packages/s

Decoding that, the test is saying it can’t find an `<h1>` element on the page. Let’s see what we can do to add that to the HTML of our home page.

Big changes to a functional test are usually a good thing to commit on their own. I failed to do so in my first draft, and I regretted it later when I changed my mind and had the change mixed up with a bunch of others. The more atomic your commits, the better:
```
$ git diff  # should show changes to functional_tests.py
$ git commit -am "Functional test now checks we can input a to-do item"
```

## The "Don't Test Constants" Rule, and Templates to the Rescue

Let’s take a look at our unit tests, lists/tests.py. Currently we’re looking for specific HTML strings, but that’s not a particularly efficient way of testing HTML. In general, one of the rules of unit testing is Don’t test constants, and testing HTML as text is a lot like testing a constant.

In other words, if you have some code that says:
```python
wibble = 3
```
There’s not much point in a test that says:
```
from myprogram import wibble
assert wibble == 3
```
Unit tests are really about testing logic, flow control, and configuration. Making assertions about exactly what sequence of characters we have in our HTML strings isn’t doing that.

What’s more, mangling raw strings in Python really isn’t a great way of dealing with HTML. There’s a much better solution, which is to use templates. Quite apart from anything else, if we can keep HTML to one side in a file whose name ends in .html, we’ll get better syntax highlighting! There are lots of Python templating frameworks out there, and Django has its own which works very well. Let’s use that.

## Refactoring to Use a Template

What we want to do now is make our view function return exactly the same HTML, but just using a different process. That’s a refactor—when we try to improve the code without changing its functionality.

That last bit is really important. If you try and add new functionality at the same time as refactoring, you’re much more likely to run into trouble. Refactoring is actually a whole discipline in itself, and it even has a reference book: Martin Fowler’s Refactoring.

The first rule is that you can’t refactor without tests. Thankfully, we’re doing TDD, so we’re way ahead of the game. Let’s check our tests pass; they will be what makes sure that our refactoring is behaviour preserving:

In [10]:
!python3 manage.py test

Creating test database for alias 'default'...
..
----------------------------------------------------------------------
Ran 2 tests in 0.002s

OK
Destroying test database for alias 'default'...


Great! We’ll start by taking our HTML string and putting it into its own file. Create a directory called lists/templates to keep templates in, and then open a file at lists/templates/home.html, to which we’ll transfer our HTML:

In [13]:
#!mkdir lists/templates

In [14]:
%%writefile lists/templates/home.html

<html>
    <title>To-Do lists</title>
</html>

Writing lists/templates/home.html


Now to change our views

In [16]:
# %load lists/views.py

from django.shortcuts import render

# Create your views here.
def home_page(request):
    return render(request, 'home.html')

Instead of building our own HttpResponse, we now use the Django render function. It takes the request as its first parameter (for reasons we’ll go into later) and the name of the template to render. Django will automatically search folders called templates inside any of your apps' directories. Then it builds an HttpResponse for you, based on the content of the template.

>Templates are a very powerful feature of Django’s, and their main strength consists of substituting Python variables into HTML text. We’re not using this feature yet, but we will in future chapters. That’s why we use `render` and (later) `render_to_string` rather than, say, manually reading the file from disk with the built-in open.

# TODO : http://chimera.labs.oreilly.com/books/1234000000754/ch04.html#_refactoring_to_use_a_template